In [1]:
import pickle

with open ('train_without_vali_test_model_3(GLOVES)_soreo-ou_test', 'rb') as f:
    model = pickle.load(f)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
import os
import random
from glob import glob

import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras import preprocessing

%matplotlib inline

width = 150
height = 150

class_names = ['soreo', 'soreya', 'roshuei', 'au', 'ae', 'ou']

camera = cv2.VideoCapture(0)
camera_height = 500

# camera.set(cv2.CAP_PROP_FPS, 1)
# fps = int(camera.get(5))
# print("fps:", fps)

# light_pink = (76, 0, 153)
# dark_pink = (204, 153, 255)
light_purp = (100, 50, 200)

dark_purp = (204, 153, 255)

while(True):
    # read a new frame
    _, frame = camera.read()
    
    # flip the frameq
    frame = cv2.flip(frame, 1)

    # rescaling camera output
    aspect = frame.shape[1] / float(frame.shape[0])
    res = int(aspect * camera_height) # landscape orientation - wide image
    frame = cv2.resize(frame, (res, camera_height))
    
    #edited part start from here:
    frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame3 = cv2.cvtColor(frame2, cv2.COLOR_RGB2HSV)
    frame4 = cv2.inRange(frame3, light_purp, dark_purp)
    frame = cv2.bitwise_and(frame, frame, mask = frame4)
    
    

    # add rectangle
    cv2.rectangle(frame, (300, 75), (650, 425), (240, 100, 0), 2)

    # get ROI
    roi = frame[75+2:425-2, 300+2:650-2]
    
    # parse BRG to RGB
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

    # resize
    roi = cv2.resize(roi, (width, height))
    
    # predict!
    roi_X = np.expand_dims(roi, axis=0)

    predictions = model.predict(roi_X)
    type_1_pred, type_2_pred, type_3_pred, type_4_pred, type_5_pred, type_6_pred = predictions[0]

    # add text
    type_1_text = '{}: {}%'.format(class_names[0], int(type_1_pred*100))
    cv2.putText(frame, type_1_text, (120, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 150, 255), 2)

    # add text
    tipe_2_text = '{}: {}%'.format(class_names[1], int(type_2_pred*100))
    cv2.putText(frame, tipe_2_text, (120, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 240, 0), 2)

    
    # add text
    tipe_3_text = '{}: {}%'.format(class_names[2], int(type_3_pred*100))
    cv2.putText(frame, tipe_3_text, (120, 130), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 0, 0), 2)

    # add text
    tipe_4_text = '{}: {}%'.format(class_names[3], int(type_4_pred*100))
    cv2.putText(frame, tipe_4_text, (120, 160), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 240), 2)
    
    # add text
    tipe_5_text = '{}: {}%'.format(class_names[4], int(type_5_pred*100))
    cv2.putText(frame, tipe_5_text, (120, 190), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (76, 0, 153), 2)
    
    
    # add text
    tipe_6_text = '{}: {}%'.format(class_names[5], int(type_6_pred*100))
    cv2.putText(frame, tipe_6_text, (120, 210), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (204, 153, 255), 2)
    
    # show the frame
    cv2.imshow("Test out", frame)

    key = cv2.waitKey(1000)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()